# Introduction to Segmentation

In this exercise we will look at different segmentation algorithms.

##  Clustering

One of the basic unsupervised learning approaches is clustering, where the algorithm has to find out which part of the data belongs together in one cluster. 

For segmentation, this data is taken from the pixel values, possible from a neighborhood of the pixels and often by calculating specific features in this region. However, for the moment we will use the raw (color) data of the pixels and possibly also the location and look at features later.

In [ ]:
import numpy as np
import skimage
import skimage.data
import skimage.io
import skimage.future
import skimage.segmentation
import sklearn.cluster
import cv2

# for displaying images in jupyter
from matplotlib import pyplot as plt

%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10, 10]
#mpl.rcParams['figure.dpi']= 400




## Exercise 1: Using KMeans for background subtraction on a medical image

In the first exercise, we want to extract the background from an image showning the hands on green background. The image is a sample from out project to detect eczema, however we will only be concerned about finding the background and the hands.

<img src=images/hand_green_small.png width=400px>

This could also be achieved by defining a suitable range of color for either the background or the foreground and use thresholding. We want to find out, if it is also possible using k-Means.

Use kmeans for clustering all the pixels in the image into 2 clusters using only the RGB values.

If we use the raw pixels, the position of the pixels do not matter. You will have to reshape the image into a 1D Array of RGB values and convert them to float.

There are different implementations of kmeans available, most noteworthy an opencv and a sklearn implementation. 

https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html


In [ ]:
image  = skimage.io.imread('images/hand_green_small.png')
plt.imshow(image)

In [ ]:
# Use k-means to find 2 clusters and display them
# For displaying you can use either the found cluster centers as colors or overlay the labels, see skimage.color.label2rgb

In [ ]:
# Solution
data = image.reshape((-1,3)).astype(np.float32)
kmeans = sklearn.cluster.KMeans(n_clusters=2)
kmeans.fit(data)
center_colors = np.uint8(kmeans.cluster_centers_)
labels_image = kmeans.labels_.reshape((image.shape[0], image.shape[1]))
result = center_colors[labels_image]
plt.imshow(result)

### Further questions

How can you find out, if the segmentation was good or not?

In [ ]:
# Calculate a value that indicates the result of the k-means algorithm.
print(kmeans.inertia_)

## Exercise 2: K-Means with multiple labels.

Try the K-Means Algorithmus on the following image to seperate the different colored figures and the gray and black background.

<img src=images/carcassonne_figures.jpg width=400px>

In [ ]:
image = skimage.io.imread('images/carcassonne_figures.jpg')
data = image.reshape((-1,3)).astype(np.float32)
plt.rcParams['figure.figsize'] = [10, 10]
plt.imshow(image)

In [ ]:
# Solution

kmeans = sklearn.cluster.KMeans(n_clusters=8)
kmeans.fit(data)
center_colors = np.uint8(kmeans.cluster_centers_)
labels_image = kmeans.labels_.reshape((image.shape[0], image.shape[1]))
result = center_colors[labels_image]
plt.rcParams['figure.figsize'] = [14, 8]
plt.subplot(1, 2, 1)
plt.imshow(result)
plt.subplot(1, 2, 2)
out = skimage.color.label2rgb(labels_image, image, kind='avg')
plt.imshow(out)

### Further questions

Try different values for the number of clusters. What results do you get?

One for the problems is that similar colors with different intensity will end up in different clusters. How could you solve that issue?

In [ ]:
# Use a different color space for k-Means
image_lab = skimage.color.rgb2lab(image)
plt.rcParams['figure.figsize'] = [12, 20]
plt.subplot(3, 1, 1)
plt.imshow(image_lab[:,:,0], cmap='gray')
print(np.max(image_lab[:,:,0]))
plt.subplot(3, 1, 2)
plt.imshow(image_lab[:,:,1], cmap='gray')
print(np.max(image_lab[:,:,1]))
plt.subplot(3, 1, 3)
plt.imshow(image_lab[:,:,2], cmap='gray')
print(np.max(image_lab[:,:,2]))

In [ ]:
# create the data for kmeans and display it as a scatter plot first
data_2d = image_lab[:,:,1:3]
data = data_2d.reshape((-1,2))
plt.rcParams['figure.figsize'] = [12, 12]
plt.plot(data[:,0], data[:,1], '.')
np.max(data[:,0])
np.max(data[:,1])

In [ ]:
kmeans = sklearn.cluster.KMeans(n_clusters=5)
kmeans.fit(data)
kmeans.cluster_centers_
labels_image = kmeans.labels_.reshape((image.shape[0], image.shape[1]))
plt.imshow(skimage.color.label2rgb(labels_image, image, kind='avg'))

In [ ]:
plt.scatter(x=data[:,0], y=data[:,1], c=kmeans.labels_)

## Exercise 3: Different segmentations for autonoumous vehicels.  

We want to try out different segmentation algorithms on a more difficult image from a dataset for semantic segmentation for autonomous vehicles:

<img src=images/000041_10.png width=400px>

### Exercise 3.1. k-Means

Try K-Means on this data set.

In [ ]:
image = skimage.io.imread('images/000041_10.png')
data = image.reshape((-1,3)).astype(np.float32)
plt.imshow(image)

In [ ]:
# Solution
kmeans = sklearn.cluster.KMeans(n_clusters=5)
kmeans.fit(data)
center_colors = np.uint8(kmeans.cluster_centers_)
labels_image = kmeans.labels_.reshape((image.shape[0], image.shape[1]))
result = center_colors[labels_image]
plt.imshow(result)

### Exercise 3.2: Superpixel segmentation

The skimage library has methods that directly segment the images into clusters using both the pixel coordinates and the color. Such methods are often used as a preliminary processing step of clustering an image into so called superpixel which can then be input into a further, often graph based, segmentation algorithm.

Try this segmentation using 
http://scikit-image.org/docs/dev/api/skimage.segmentation.html#skimage.segmentation.slic



In [ ]:
labels = skimage.segmentation.slic(image, n_segments=1000, compactness=10, multichannel=True)
print(labels.shape)
print(np.max(labels))
plt.imshow(skimage.color.label2rgb(labels, image, kind='avg'))

### Exercise 3.3: GraphCut

The skimage packages contains a graph cut algorithms to segment a (superpixel) image using the graph cut algorithms.

To use is, first the *Region Adjacency Graph* of the segmented superpixels must be constructed:

http://scikit-image.org/docs/dev/api/skimage.future.graph.html#skimage.future.graph.rag_mean_color

and then the graph cut algorithms can be applied:

http://scikit-image.org/docs/dev/api/skimage.future.graph.html

Use both methods to segment the result of the superpixel segmentation using graph cuts and display the result.


In [ ]:
# Solution

graph = skimage.future.graph.rag_mean_color(image, labels, mode='similarity')
labels2 = skimage.future.graph.cut_normalized(labels, graph, in_place=False)
#print(labels)
#print(labels2)
plt.imshow(skimage.color.label2rgb(labels2, image, kind='avg'))

In [ ]:
skimage.future.graph.show_rag(labels, graph, image)

### Exercise 3.4: Mean Shift Segmentation

OpenCV contains a method to directly calculate a mean shift segmentation from an image:

https://docs.opencv.org/3.4/d4/d86/group__imgproc__filter.html

Apply this method on the (original) image above and evaluate the result. Check the different parameter settings of the algorithm.



In [ ]:
criteria = (cv2.TERM_CRITERIA_EPS, 1000, 0.001)
mean_shift_image = cv2.pyrMeanShiftFiltering(image, sp=20.0, sr=50.0, maxLevel=1, termcrit=criteria)
plt.imshow(mean_shift_image)

## Exercise 4 (optional): k-Means with position and color

Implement a k-Means clustering that uses the position of a pixel together with the color for clustering.


In [ ]:
image = skimage.io.imread('images/carcassonne_figures.jpg')
data_col = image.reshape((-1,3)).astype(np.float32) / 255.0
idx = np.indices((image.shape[0], image.shape[1]))
print(np.max(data_col))

In [ ]:
idx = np.indices((image.shape[0], image.shape[1]))
idx_all = np.stack([idx[0,:,:],idx[1,:,:]], axis=2)
factor = 1.0
data_idx = idx_all.reshape((-1,2)).astype(np.float32) * factor / np.max(idx_all)
print(np.max(data_idx))

In [ ]:
print(data_col.shape)
print(data_idx.shape)
data = np.concatenate([data_col, data_idx], axis=1)
print(data.shape)

In [ ]:
kmeans = sklearn.cluster.KMeans(n_clusters=11)
kmeans.fit(data)
kmeans.cluster_centers_
labels_image = kmeans.labels_.reshape((image.shape[0], image.shape[1]))
plt.imshow(skimage.color.label2rgb(labels_image))